In [1]:
"""
The first step: to lift the data (tabular data at the end) from one of these text files. 
e.g. https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/bhattacharya2018/bhattacharya2018jpc56.txt
"""

"""
Couple of things to make progress:
1. 
1) For the table lifting, remove the ‘#’ in front
2) differentiate between the two types of table: the chronological information and the paleo information.
   -> choronological if the column name of the table has 'age'

"""   

import requests
import pandas as pd
import numpy as np

def getTable (url):
    txt_data = requests.get(url).text
    txt_data = txt_data.replace('\r', '')  # remove "\r" in text
    splited_data = txt_data.split('\n')   
    length = len(splited_data)

    
    """
    tabular data: 1. tab seperated (exception) 2. the number of columns is the same in the table.     
    """
    
    # remove the ‘#’ in front
    for i in range(length):    
        if splited_data[i].startswith("#"):
            splited_data[i] = splited_data[i][1:].lstrip()
    
    # to get the number of columns
    # count # of tab seperated words, otherwise white space seperated # of words for exception 
       
    table_index = list()  # [[start_index, end_index],..]
    
    i = 0
    while (i<length):            
        if "\t" in splited_data[i]:
            num = len(splited_data[i].split('\t'))
            if num > 2:
                start_index = i  # candidate of the first row of the table
                end_index = i
                i += 1            
                while (i<length):                
                    if len(splited_data[i].split('\t')) == num:
                        end_index = i
                        i +=1
                        
                    else:
                        break
                if (end_index-start_index) > 2:
                    table_index.append([start_index, end_index])
            else:
                i += 1
                    
        else:  # in case of white space seperated table
            num = len(splited_data[i].split())
            if num > 2:
                start_index = i  # candidate of the first row of the table
                end_index = i            
                
                i += 1
                while (i<length):                
                    if len(splited_data[i].split()) == num:
                        end_index = i
                        i +=1                        
                    else:
                        break
                if end_index-start_index > 2:
                    table_index.append([start_index, end_index])
            else:
                i += 1   
    
    # get tabular data  
    
    # differentiate types of table: chronological and paleo information      
    chron_tables = []
    paleo_tables = []
    
    for start_index, end_index in table_index:
        
        table = list()
        if '\t' in splited_data[start_index]:
            for i in range(start_index, end_index+1):
                table.append(splited_data[i].split('\t'))
                
        else: 
            for i in range(start_index, end_index+1):
                table.append(splited_data[i].split())

                
        # differentiate types of table
        # btw "----" and the table, word "chronology" exists, then chronological
        
        missing_value = False 
        flag = True  # default: paleo info flag
        for k in range(start_index-1, 0, -1):
            if "-----" in splited_data[k]: 
                break

            lower = splited_data[k].lower()
            if "chronology" in lower: 
                flag = False

            # missing value detect      
            if "missing" in lower and "value" in lower:
                missing_value = lower.split(":")[-1].strip()                   

            # missing value => change to nan
            if missing_value != False:
                for row_i in range(1, len(table)):
                    for column_i in range(len(table[0])):
                        if table[row_i][column_i].lower() == missing_value:
                            table[row_i][column_i] = 'nan'
                            
                        else:               
                            try:
                                missing_value2 = float(missing_value)
                                num2 = float(table[row_i][column_i])

                                if num2 == missing_value2:  
                                    table[row_i][column_i] = 'nan'
                            except:
                                continue    
                
            
        if flag:
            paleo_tables.append(table)
        else:
            chron_tables.append(table)   
       
        
    # convert dataframe
    df_list = []
    for table in paleo_tables:
        df = pd.DataFrame(np.array(table[1:]), columns=table[0])           
        df = df.style.set_caption('Paleo Information')
        df_list.append(df)            

    for table in chron_tables:
        df = pd.DataFrame(np.array(table[1:]), columns=table[0]) 
        df = df.style.set_caption('Chronological Information')
        df_list.append(df)
            
    return df_list

In [2]:
url1 = "https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/bhattacharya2018/bhattacharya2018jpc56.txt"
url2 = "https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/khider2014/khider2014-raw.txt"
url3 = "https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/khider2011/khider2011.txt"

# https://www.ncdc.noaa.gov/paleo-search/study/search.json?NOAAStudyId=24890
# https://www.ncdc.noaa.gov/paleo-search/study/search.json?NOAAStudyId=16055
# https://www.ncdc.noaa.gov/paleo-search/study/search.json?NOAAStudyId=18315

In [3]:
df_list1 = getTable(url1)
print(len(df_list1))

1


In [4]:
df_list1[0]

,depth_cm,age_calBP,dDwax,dDwax-iv,d13Cwax,percentJAS,JASerror
0,512,9798.6,-147.94,-148.94,-26.429,77.587,7.8528
1,537,9899.6,-150.36,-151.41,-26.313,73.239,7.8871
2,593,10134,-147.32,-148.47,-26.717,76.986,8.1359
3,609,10197,-149.57,-150.73,-26.118,74.624,8.0159
4,658,10395,-148.86,-150.08,-25.753,77.187,8.0041
5,701,10593,-150.65,-151.93,-26.282,72.452,7.8048
6,741,10789,-156.35,-157.68,-26.004,62.068,8.0316
7,762,10889,-152.29,-153.66,-26.508,68.636,7.8856
8,823,11189,-154.04,-155.67,-26.63,63.91,8.0646
9,876,11461,-148.75,-150.72,-26.35,74.31,8.0889


In [5]:
df_list2 = getTable(url2)
print(len(df_list2))

2


In [6]:
df_list2[0]

,depth_cm,Mg/Ca-g.rub-w,d18Og.rub-w,d13Cg.rub-w,d18Oc.mund,d13Cc.mund,notes
0,1,5.21,nan,nan,nan,nan,"data from Stott et al. (2004, 2007)"
1,1,nan,nan,nan,-0.30,2.45,
2,2.5,5.04,0.76,-2.83,nan,nan,
3,3,nan,1.53,-2.81,nan,nan,"data from Stott et al. (2004, 2007)"
4,4,nan,0.90,-2.78,nan,nan,
5,5,5.10,0.75,-2.64,0.03,2.77,
6,5,nan,0.95,-2.93,nan,nan,
7,6.5,5.04,0.74,-2.85,nan,nan,
8,7,5.01,0.92,-2.57,nan,nan,"data from Stott et al. (2004, 2007)"
9,7,nan,nan,nan,-0.27,2.67,


In [7]:
df_list2[1]

,Depth,Species,CAMS#,14CageÂ±1Ï,Calendar AgeÂ±1Ï
0,12cm,Gs,OS-36493,580Â±110,269Â±222
1,55cm,Gs/Gr,OS-37308,815Â±200,491Â±395
2,99cm,Gs/Gr,OS-37292,1010Â±100,688Â±205
3,145cm,Gs,OS-37306,1090Â±120,742Â±242
4,201cm,MP,90357,1750Â±15,1389Â±149
5,238cm,Gs,OS-36485,1900Â±100,1551Â±262
6,351cm,Gs/Gr,104528,2575Â±20,2368Â±232
7,402cm,Gs,OS-37289,3960Â±150,4069Â±433
8,501cm,Gs,OS-37303,4070Â±190,4245Â±530
9,610cm,Gs/Gr,104529,4925Â±30,5344Â±216


In [8]:
df_list3 = getTable(url3)
print(len(df_list3))

2


In [9]:
df_list3[0]

,depth_top,depth_bottom,age_ADbot,age_ADtop,d13CcarbVPDB,d18OcarbVPDB,wgt-ind
0,0,1,1843,1851,0.936,-2.254,34
1,0,1,1843,1851,0.895,-2.356,37
2,0,1,1843,1851,0.514,-2.63,20
3,0,1,1843,1851,0.9,-2.48,25
4,0,1,1843,1851,0.957,-2.094,29
5,0,1,1843,1851,0.754,-2.664,28
6,0,1,1843,1851,1.023,-2.147,19
7,0,1,1843,1851,0.844,-1.862,13
8,0,1,1843,1851,1.027,-2.202,35
9,0,1,1843,1851,0.376,-2.012,18


In [10]:
df_list3[1]

,Depth,14C.raw,14C.raw_err,Labcode,Calib.14C,Calib_14C_sigma,calib_method,reservoir,Reservoir_err
0,0,580,45,CAMS-95299,98,51,CALIB6.0,74,70
1,12,395,90,OS-38302,>modern,N/A,CALIB6.0,74,70
2,50,1110,60,CAMS-1000234,600,69.5,CALIB6.0,74,70
3,94,1745,45,CAMS-100235,1220,82.5,CALIB6.0,74,70
4,109,1870,110,OS-38335,1366,135.5,CALIB6.0,74,70
5,130,2260,45,CAMS-100236,1782,100.5,CALIB6.0,74,70


In [11]:
"""
The second step is to lift some of the metadata so it would be nice if you could learn how to use their API: 

https://www.ncdc.noaa.gov/paleo-search/api

If you use the Study ID from above, what can you retrieve?
"""

import re

# get metadata from extracted study ID
def getMetaData (url):
    txt_data = requests.get(url).text
    splited_data = txt_data.split('\n')
 
   
    # find "study" to get study id: 5 digits after "study"
    study_lines = []
    for line in splited_data:
        if "study" in line.lower():
            study_lines.append(line.lower().partition("study")[2])
            
    study_id = getStudyID(study_lines)
    
    api = "https://www.ncdc.noaa.gov/paleo-search/study/search.json?NOAAStudyId=" + study_id
    
    return requests.get(api).json()

            
def getStudyID(study_lines):            
    for line in study_lines:
        temp = re.search('\d{% s}'% 5, line)
        study_id = (temp.group(0) if temp else '')
        if study_id:
            return study_id        

In [12]:
getMetaData (url1)

{'study': [{'xmlId': '23014',
   'NOAAStudyId': '24890',
   'studyName': 'Guaymas Basin 9-12kYrBP Leaf Wax Isotopes and Precipitation Reconstructions',
   'doi': None,
   'uuid': 'ec9a1fdf-f98e-4132-adf2-4243b25d4594',
   'dataPublisher': 'NOAA',
   'contactInfo': {'type': 'CONTACT INFORMATION',
    'shortName': 'DOC/NOAA/NESDIS/NCEI',
    'longName': 'National Centers for Environmental Information, NESDIS, NOAA, U.S. Department of Commerce ',
    'address': '325 Broadway, E/NE31',
    'city': 'Boulder',
    'state': 'CO',
    'postalCode': '80305-3328',
    'country': 'USA',
    'dataCenterUrl': 'https://www.ncdc.noaa.gov/data-access/paleoclimatology-data',
    'email': 'paleo@noaa.gov',
    'phone': '303-497-6280',
    'fax': '303-497-6513',
    'constraints': 'Please cite original publication, online resource, dataset and publication DOIs (where available), and date accessed when using downloaded data. If there is no publication information, please cite investigator, title, online r

In [13]:
getMetaData (url2)
            

{'study': [{'xmlId': '13818',
   'NOAAStudyId': '16055',
   'studyName': 'Western Tropical Pacific SST and Isotope Data From MD98-2181 During the Holocene',
   'doi': None,
   'uuid': '02ef0ad2-716a-4e3a-a0c3-db797d27ffe7',
   'dataPublisher': 'NOAA',
   'contactInfo': {'type': 'CONTACT INFORMATION',
    'shortName': 'DOC/NOAA/NESDIS/NCEI',
    'longName': 'National Centers for Environmental Information, NESDIS, NOAA, U.S. Department of Commerce ',
    'address': '325 Broadway, E/NE31',
    'city': 'Boulder',
    'state': 'CO',
    'postalCode': '80305-3328',
    'country': 'USA',
    'dataCenterUrl': 'https://www.ncdc.noaa.gov/data-access/paleoclimatology-data',
    'email': 'paleo@noaa.gov',
    'phone': '303-497-6280',
    'fax': '303-497-6513',
    'constraints': 'Please cite original publication, online resource, dataset and publication DOIs (where available), and date accessed when using downloaded data. If there is no publication information, please cite investigator, title, onl

In [14]:
getMetaData (url3)

{'study': [{'xmlId': '16017',
   'NOAAStudyId': '18315',
   'studyName': 'Makassar Strait - Single specimens of P. obliquiloculata d18O and d13C from 704-1851 AD',
   'doi': None,
   'uuid': 'd8465979-11bb-47c0-9bc9-ac55f38077e4',
   'dataPublisher': 'NOAA',
   'contactInfo': {'type': 'CONTACT INFORMATION',
    'shortName': 'DOC/NOAA/NESDIS/NCEI',
    'longName': 'National Centers for Environmental Information, NESDIS, NOAA, U.S. Department of Commerce ',
    'address': '325 Broadway, E/NE31',
    'city': 'Boulder',
    'state': 'CO',
    'postalCode': '80305-3328',
    'country': 'USA',
    'dataCenterUrl': 'https://www.ncdc.noaa.gov/data-access/paleoclimatology-data',
    'email': 'paleo@noaa.gov',
    'phone': '303-497-6280',
    'fax': '303-497-6513',
    'constraints': 'Please cite original publication, online resource, dataset and publication DOIs (where available), and date accessed when using downloaded data. If there is no publication information, please cite investigator, tit